# **Домашнее задание: API-сервис сокращения ссылок.**

## **📌 Контекст задачи**:

Вам предстоит разработать сервис, который позволяет пользователям сокращать длинные ссылки, получать их аналитику и управлять ими. Основная идея — пользователь вводит длинный URL, а ваш сервис генерирует для него короткую ссылку, которую можно использовать для быстрого доступа.

**Как это работает:**

- Пользователь отправляет запрос (`POST /links/shorten`) с длинной ссылкой.

- Сервис генерирует уникальный короткий код и возвращает его пользователю.

- При открытии короткой ссылки (`GET-запрос к /{short_code}`) сервис ищет в базе данных соответствующий оригинальный URL и перенаправляет пользователя (Redirect).

**Примеры аналогичных сервисов:**
- [tinyURL](https://tinyurl.com)
- [bitly](https://bitly.com/)

In [1]:
!pip3 install -r requirements-test.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 333 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 207 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 6.0 MB 7.8 MB/s eta 0:00:01
  distutils: /private/var/folders/2w/1s7n38315rx8pkdpz8t33nj00000gn/T/pip-build-env-t4p_kree/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/2w/1s7n38315rx8pkdpz8t33nj00000gn/T/pip-build-env-t4p_kree/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/2w/1s7n38315rx8pkdpz8t33nj00000gn/T/pip-build-env-t4p_kree/normal'
  distutils: /private

In [9]:
!pytest

ImportError while loading conftest '/Users/kostiks/study/ai_masters/ai_python/fastapi_project/tests/conftest.py'.
tests/conftest.py:16: in <module>
    from src.application.main import app
src/application/main.py:9: in <module>
    Base.metadata.create_all(bind=engine)
/Library/Python/3.9/site-packages/sqlalchemy/sql/schema.py:5825: in create_all
    bind._run_ddl_visitor(
/Library/Python/3.9/site-packages/sqlalchemy/engine/base.py:3253: in _run_ddl_visitor
    with self.begin() as conn:
/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:117: in __enter__
    return next(self.gen)
/Library/Python/3.9/site-packages/sqlalchemy/engine/base.py:3243: in begin
    with self.connect() as conn:
/Library/Python/3.9/site-packages/sqlalchemy/engine/base.py:3279: in connect
    return self._connection_cls(self)
/Library/Python/3.9/site-packages/sqlalchemy/engine/base.py:147: in __init__
    Connection._handle_dbapi_exception_noc

In [8]:
!pip3 install passlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 525 kB 4.5 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


---

## **🔴 Функционал сервиса:**

> Ниже представлены функции - 5 обязательных и несколько дополнительных (можно придумать свои).

### **⭕ Обязательные функции:**
1. **Создание / удаление / изменение / получение информации по короткой ссылке:**
  - `POST /links/shorten` – создает короткую ссылку.
  - `GET /links/{short_code}` – перенаправляет на оригинальный URL.
  - `DELETE /links/{short_code}` – удаляет связь.
  - `PUT /links/{short_code}` – обновляет URL (То есть, короткий адрес. Будем засчитывать и другую реализацию - когда к короткой ссылке привязывается новая длинная).
2. **Статистика по ссылке:**
  - `GET /links/{short_code}/stats`
  - Отображает оригинальный URL, возвращает дату создания, количество переходов, дату последнего использовани.
3. **Создание кастомных ссылок (уникальный alias):**
  - `POST /links/shorten` (с передачей `custom_alias`).
  - Важно проверить уникальность `alias`.
4. **Поиск ссылки по оригинальному URL:**
  - `GET /links/search?original_url={url}`
5. **Указание времени жизни ссылки:**
  - `POST /links/shorten` (создается с параметром `expires_at` в формате даты с точностью до минуты).
  - После указанного времени короткая ссылка автоматически удаляется.



### **⭕ Дополнительные функции:**

> Каждый метод должна быть отличим от других по логике и функционалу. Вы можете придумать свой вариант, но ниже будет пару наших идей.

- **Удаление неиспользуемых ссылок:**
  - К примеру, спустя по умолчанию спустя `N` дней после последнего перехода по ссылке.
  - `N` задается для всех ссылок разом.
- Отображение истории всех истекших ссылок с информацией о них.
- Группировка ссылок по проектам.
- Создание коротких ссылок для незарегистрированных пользователей.


### **⭕ Регистрация:**
Так как пользователь должен иметь возможность управлять ссылками, необходимо сделать простую регистрацию.

Изменение и удаление ссылки должно быть доступно только зарегистрированным пользователям, в то время как `GET` / `POST` - всем. Чтобы это поддерживать - храним информацию о том какой пользователь создавал ссылку (залогиненный или нет) и кладем эту информацию в базу данных.

### **⭕ База данных и кэширование:**

- Используйте БД (к примеру, `PostgreSQL`) как основное хранилище ссылок, так как позволяет легко обновлять, удалять и искать данные.

- **Обязательно используйте `Redis` или аналоги** для кэширования тех данных, где это нужно и полезно (к примеру, кэширование самых популярных ссылок). **Наличие кэширования для endpoint-ов будет оцениваться.**

- Не забудьте настроить очистку кэша при обновлении / удалении ссылки.


---

## **🔴 Формат сдачи работы:**

> Работа сдается в AnyTask.

- Ссылка на код на `GitHub`, включая `docker` или `docker-compose`.
- Cсылку на развернутый работающий сервис (можете использовать этот [хостинг](https://dashboard.render.com/)).

- `README.md` в репозитории, содержащий:
  * Описание API.
  * Примеры запросов.
  * Инструкцию по запуску.
  * Описание БД (при наличии).


---

## **🔴 Критерии оценки:**

- 5 Обязательных функций - до `7.5` баллов (`1.5` каждая)
- Наличие кэширования важных эндпоинтов - `1` балл.
- 2 Дополнительные функции - до `1.5` баллов (по `0.75` каждая).
- При реализации большей функциональности, нестандартных корректных подходах и т.п. оценивание **на усмотрение** проверяющего.

Всего за задание не может быть больше 14 баллов.
